In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import io
import statsmodels.formula.api as smf
import statsmodels.api as sm 
import scipy
import scipy.stats as stats
import re
import matplotlib.pyplot as plt
from statsmodels.iolib.summary2 import summary_params # вывод результатов тестирования
from statsmodels.iolib.summary2 import summary_col # вывод результатов тестирования
from statsmodels.stats.outliers_influence import variance_inflation_factor # VIF

# 1 Тесты

## 1.1 sleep equation

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/ryupepa/Econometrica/master/python-notebooks/data-csv/sleep75.csv')

In [3]:
model = smf.ols(data=df, formula='sleep~totwrk+age+I(age**2)+male+smsa+south').fit()

In [4]:
model.resid

0       47.889915
1       23.947379
2     -468.006774
3      168.487279
4      103.053368
          ...    
701   -198.194706
702    -11.720249
703    262.430763
704   -419.513088
705   -281.450578
Length: 706, dtype: float64

In [5]:
# Для того, чтобы проверить наличие гетероскедастичности с помощью теста Бройша-Пагана,
# необходимо построить вспомогательную регрессию,
#зависимой переменной в которой будут квадраты остатков исходной регрессии.

In [6]:
# квадраты остатков добавим в датафрейм, воспользовавшись методом '.resid', возвращающий значения остатков
df['res2']=model.resid**2
df['res']=model.resid

In [7]:
df

,age,black,case,clerical,construc,educ,earns74,gdhlth,inlf,leis1,...,union,worknrm,workscnd,exper,yngkid,yrsmarr,hrwage,agesq,res2,res
0,32,0,1,0.000000,0.000000,12,0,0,1,3529,...,0,3438,0,14,0,13,7.070004,1024,2293.443965,47.889915
1,31,0,2,0.000000,0.000000,14,9500,1,1,2140,...,0,5020,0,11,0,0,1.429999,961,573.476946,23.947379
2,44,0,3,0.000000,0.000000,17,42500,1,1,4595,...,0,2815,0,21,0,0,20.529997,1936,219030.340655,-468.006774
3,30,0,4,0.000000,0.000000,12,42500,1,1,3211,...,0,3786,0,12,0,12,9.619998,900,28387.963146,168.487279
4,64,0,5,0.000000,0.000000,14,2500,1,1,4052,...,0,2580,0,44,0,33,2.750000,4096,10619.996605,103.053368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
701,45,0,702,0.182331,0.030075,12,5500,1,0,5069,...,0,2026,0,27,0,18,NaN,2025,39281.141585,-198.194706
702,34,0,703,0.182331,0.030075,10,2500,0,0,5885,...,1,465,210,18,0,4,NaN,1156,137.364246,-11.720249
703,37,0,704,0.182331,0.030075,12,3500,1,0,4719,...,0,1851,0,19,0,17,NaN,1369,68869.905163,262.430763
704,54,0,705,0.182331,0.030075,17,32500,1,0,5149,...,1,1481,480,31,0,22,NaN,2916,175991.230735,-419.513088


In [8]:
 sub_model = smf.ols(data=df, formula='res2~totwrk+age+I(age**2)+male+smsa+south').fit()

In [9]:
# наблюдаемое значение
np.round(len(df)*sub_model.rsquared, 2) #Len(df) - количество измерений

8.31

In [10]:
# критическое значение
Hi2 = stats.chi2.ppf(1-0.05,6)
Hi2

12.591587243743977

In [11]:
# значимо - гетероскедастичность 
# не значимо - гомоскедастичность

## 1.2 wage equation

In [32]:
df = pd.read_csv('https://raw.githubusercontent.com/ryupepa/Econometrica/master/python-notebooks/data-csv/wage1.csv')
df

,wage,educ,exper,tenure,nonwhite,female,married,numdep,smsa,northcen,...,trcommpu,trade,services,profserv,profocc,clerocc,servocc,lwage,expersq,tenursq
0,3.10,11,2,0,0,1,0,2,1,0,...,0,0,0,0,0,0,0,1.131402,4,0
1,3.24,12,22,2,0,1,1,3,1,0,...,0,0,1,0,0,0,1,1.175573,484,4
2,3.00,11,2,0,0,0,0,2,0,0,...,0,1,0,0,0,0,0,1.098612,4,0
3,6.00,8,44,28,0,0,1,0,1,0,...,0,0,0,0,0,1,0,1.791759,1936,784
4,5.30,12,7,2,0,0,1,1,0,0,...,0,0,0,0,0,0,0,1.667707,49,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,15.00,16,14,2,0,1,1,2,0,0,...,0,0,0,1,1,0,0,2.708050,196,4
522,2.27,10,2,0,0,1,0,3,0,0,...,0,1,0,0,1,0,0,0.819780,4,0
523,4.67,15,13,18,0,0,1,3,0,0,...,0,0,0,0,1,0,0,1.541159,169,324
524,11.56,16,5,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,2.447551,25,1


In [33]:
model12 = smf.ols(data=df, formula='np.log(wage)~exper+I(exper**2)+female+married+smsa').fit()

In [34]:
# квадраты остатков добавим в датафрейм, воспользовавшись методом '.resid', возвращающий значения остатков
df['res2']=model.resid**2
df['res']=model.resid

In [35]:
sub_model12 = smf.ols(data=df, formula='res2~exper+I(exper**2)+female+married+smsa').fit()

In [36]:
# наблюдаемое значение
St = np.round(len(df)*sub_model12.rsquared, 2) #Len(df) - количество измерений
St # F_nabl

8.82

In [37]:
# критическое значение
Hi2 = stats.chi2.ppf(1-0.05,5)
round(Hi2,2) # F_crit

11.07

$F_{nabl}<F_{crit}$
Тест укзывает на гомоскедастичность

## 1.3 output equation

In [38]:
df = pd.read_csv('https://raw.githubusercontent.com/ryupepa/Econometrica/master/python-notebooks/data-csv/Labour.csv')
df

,capital,labour,output,wage
0,2.606563,184,9.250759,43.080307
1,1.323237,91,3.664310,27.780016
2,22.093692,426,28.781516,44.467748
3,10.737851,72,4.124642,39.734710
4,1.161365,46,2.890150,34.650709
...,...,...,...,...
564,2.625403,20,1.424376,33.477545
565,1.276386,61,2.109048,26.300732
566,1.953869,117,6.241870,41.153979
567,1.318527,46,7.902237,66.720139


In [39]:
model13 = smf.ols(data=df, formula='np.log(output)~np.log(capital)+np.log(labour)+I(np.log(capital)**2)+I(np.log(labour)**2)').fit()

In [40]:
# квадраты остатков добавим в датафрейм, воспользовавшись методом '.resid', возвращающий значения остатков
df['res2']=model.resid**2
df['res']=model.resid

In [41]:
sub_model13 = smf.ols(data=df, formula='res2~np.log(capital)+np.log(labour)+I(np.log(capital)**2)+I(np.log(labour)**2)').fit()

In [44]:
# наблюдаемое значение
St = np.round(len(df)*sub_model13.rsquared, 2) #Len(df) - количество измерений
St # F_nabl

5.54

In [45]:
# критическое значение
Hi2 = stats.chi2.ppf(1-0.05,4)
round(Hi2,2) # F_crit

9.49

$F_{nabl}>F_{crit}$
Тест укзывает на гетероскедастичность

## 1.4 cost equation #1

In [46]:
df = pd.read_csv('https://raw.githubusercontent.com/ryupepa/Econometrica/master/python-notebooks/data-csv/Electricity.csv')
df

,cost,q,pl,sl,pk,sk,pf,sf
0,0.2130,8.0,6869.47,0.3291,64.945,0.4197,18.000,0.2512
1,3.0427,869.0,8372.96,0.1030,68.227,0.2913,21.067,0.6057
2,9.4059,1412.0,7960.90,0.0891,40.692,0.1567,41.530,0.7542
3,0.7606,65.0,8971.89,0.2802,41.243,0.1282,28.539,0.5916
4,2.2587,295.0,8218.40,0.1772,71.940,0.1623,39.200,0.6606
...,...,...,...,...,...,...,...,...
153,6.8293,946.6,10642.16,0.0883,43.600,0.1914,51.463,0.7203
154,3.7605,377.0,7432.24,0.2117,74.120,0.2274,33.436,0.5609
155,3.9822,391.0,5826.04,0.1926,78.288,0.0924,44.633,0.7151
156,30.1880,5317.0,9586.63,0.0845,78.008,0.2009,41.840,0.7147


In [47]:
model14 = smf.ols(data=df, formula='np.log(cost)~np.log(q)+I(np.log(q)**2)+np.log(pl)+np.log(pk)+np.log(pf)').fit()

In [48]:
# квадраты остатков добавим в датафрейм, воспользовавшись методом '.resid', возвращающий значения остатков
df['res2']=model.resid**2
df['res']=model.resid

In [49]:
sub_model14 = smf.ols(data=df, formula='res2~np.log(q)+I(np.log(q)**2)+np.log(pl)+np.log(pk)+np.log(pf)').fit()

In [50]:
# наблюдаемое значение
St = np.round(len(df)*sub_model14.rsquared, 2) #Len(df) - количество измерений
St # F_nabl

6.09

In [51]:
# критическое значение
Hi2 = stats.chi2.ppf(1-0.05,5)
round(Hi2,2) # F_crit

11.07

$F_{nabl}>F_{crit}$
Тест укзывает на гетероскедастичность

## 1.5 cost equation #2

In [52]:
df = pd.read_csv('https://raw.githubusercontent.com/ryupepa/Econometrica/master/python-notebooks/data-csv/Electricity.csv')
df

,cost,q,pl,sl,pk,sk,pf,sf
0,0.2130,8.0,6869.47,0.3291,64.945,0.4197,18.000,0.2512
1,3.0427,869.0,8372.96,0.1030,68.227,0.2913,21.067,0.6057
2,9.4059,1412.0,7960.90,0.0891,40.692,0.1567,41.530,0.7542
3,0.7606,65.0,8971.89,0.2802,41.243,0.1282,28.539,0.5916
4,2.2587,295.0,8218.40,0.1772,71.940,0.1623,39.200,0.6606
...,...,...,...,...,...,...,...,...
153,6.8293,946.6,10642.16,0.0883,43.600,0.1914,51.463,0.7203
154,3.7605,377.0,7432.24,0.2117,74.120,0.2274,33.436,0.5609
155,3.9822,391.0,5826.04,0.1926,78.288,0.0924,44.633,0.7151
156,30.1880,5317.0,9586.63,0.0845,78.008,0.2009,41.840,0.7147


In [54]:
model15 = smf.ols(data=df, formula='np.log(cost)~np.log(q)+I(np.log(q)**2)+np.log(pl)+np.log(pk)+np.log(pf)+I(np.log(pl)**2)+I(np.log(pk)**2)+I(np.log(pf)**2)').fit()

In [55]:
# квадраты остатков добавим в датафрейм, воспользовавшись методом '.resid', возвращающий значения остатков
df['res2']=model.resid**2
df['res']=model.resid

In [56]:
sub_model15 = smf.ols(data=df, formula='res2~np.log(q)+I(np.log(q)**2)+np.log(pl)+np.log(pk)+np.log(pf)+I(np.log(pl)**2)+I(np.log(pk)**2)+I(np.log(pf)**2)').fit()

In [57]:
# наблюдаемое значение
St = np.round(len(df)*sub_model15.rsquared, 2) #Len(df) - количество измерений
St # F_nabl

14.41

In [58]:
# критическое значение
Hi2 = stats.chi2.ppf(1-0.05,8)
round(Hi2,2) # F_crit

15.51

$F_{nabl}>F_{crit}$
Тест укзывает на гетероскедастичность